In [ ]:
import numpy as np
import os
import sys
import tensorflow as tf
from distutils.version import StrictVersion
import matplotlib.pyplot as plt
from PIL import Image
from preprocessing.preprocess import pre
%matplotlib inline
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from utils import label_map_util
from utils import visualization_utils as vis_util

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')


PATH_TO_TRAINED_MODEL = '/object_detection_task/training/output_inference_graph.pb'
PATH_TO_LABELS = os.path.join('data', '/object_detection_task/training/object-detection.pbtxt')

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_TRAINED_MODEL, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

PATH_TO_TEST_IMAGES_DIR = '/object_detection_task/images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'raccoon-{}.jpg'.format(i)) for i in range(1, 3) ]
IMAGE_SIZE = (12, 8)
the_pre = pre()

for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  image_np = the_pre.load_image_into_numpy_array(image)
  image_np_expanded = np.expand_dims(image_np, axis=0)
  output_dict = the_pre.run_inference_for_single_image(image_np_expanded, detection_graph)
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)
